In [1]:
from pathlib import Path
import json
import sys

import gtfstk as gt
import pandas as pd
import numpy as np
import shapely.geometry as sg
import folium

sys.path.append('../')

import gtfs_map_matcher as mm

%load_ext autoreload
%autoreload 2

DATA_DIR = Path('../data')
OUT_DIR = Path('../output')
SECRETS_PATH = Path('../secrets.json')

In [2]:
path = DATA_DIR/'wellington_gtfs_20171016.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.assess_quality()

,indicator,value
0,num_route_short_names_duplicated,0
1,frac_route_short_names_duplicated,0
2,num_stop_time_dists_missing,0
3,frac_stop_time_dists_missing,0
4,num_direction_ids_missing,0
5,frac_direction_ids_missing,0
6,num_trips_missing_shapes,0
7,frac_trips_missing_shapes,0
8,num_departure_times_missing,0
9,frac_departure_times_missing,0


In [3]:
# Visually test spoints_trip_points()

t = feed.trips.merge(feed.routes)
t = t[t['route_short_name'].str.len() == 2].copy()
%time spoints_1 = mm.sample_trip_points(feed, t.trip_id, 5)
%time spoints_2 = mm.sample_trip_points(feed, t.trip_id, 100)
%time spoints_3 = mm.sample_trip_points(feed, t.trip_id, point_dist=0.1)

print('num stop patterns = ', len(spoints_2))
pattern = list(spoints_1.keys())[0]


CPU times: user 1.17 s, sys: 4 ms, total: 1.17 s
Wall time: 1.17 s
CPU times: user 2.28 s, sys: 8 ms, total: 2.28 s
Wall time: 2.26 s
CPU times: user 2.55 s, sys: 32 ms, total: 2.58 s
Wall time: 2.56 s
num stop patterns =  181


In [ ]:
l1 = sg.LineString(spoints_1[pattern])
l2 = sg.LineString(spoints_2[pattern])
l3 = sg.LineString(spoints_3[pattern])

# Get actual trip shape
p = mm.get_stop_patterns(feed)
shid = p.loc[p['stop_pattern'] == pattern, 'shape_id'].unique()[0]
la = feed.build_geometry_by_shape(shape_ids=[shid], use_utm=False)[shid]

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(l1), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
folium.GeoJson(sg.mapping(l2), 
  style_function=lambda x: {'color': 'orange'}).add_to(mappy)
folium.GeoJson(sg.mapping(l3), 
  style_function=lambda x: {'color': 'yellow'}).add_to(mappy)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
mappy

In [ ]:
# Map match with Mapzen

mpoints = mm.map_match_mapzen({pattern: spoints_2[pattern]},
  mm.get_secret(SECRETS_PATH, 'MAPZEN_API_KEY'))[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with OSRM

mpoints = mm.map_match_osrm({pattern: spoints_2[pattern]})[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with Mapbox, which uses OSRM

mpoints = mm.map_match_mapbox({pattern: spoints_2[pattern]},
  mm.get_secret(SECRETS_PATH, 'MAPBOX_API_KEY'))[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with Google

mpoints = mm.map_match_google({pattern: spoints_2[pattern]},
  mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'))[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Test batch map match

n = 10
print('num requests =', n)
#test = {k: v for k, v in list(spoints_2.items())[:n]}
#api_key = mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY')
api_key = mm.get_secret(SECRETS_PATH, 'MAPBOX_API_KEY')

%time mpoints = mm.map_match(test, 'mapbox', api_key)

In [ ]:
len(mpoints)

In [30]:
t = feed.trips.merge(feed.routes)
trip_id, shape_id = t.loc[t['route_type'] == 3, ['trip_id', 'shape_id']].iloc[0].values
print(trip_id, shape_id)
trip_ids = [trip_id]
%time feed2 = mm.create_shapes(feed, 'google', mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'), trip_ids=trip_ids)

290__0__7672__MADG__16__11__16__11 290:747#23138#23171
CPU times: user 2.12 s, sys: 40 ms, total: 2.16 s
Wall time: 2.91 s


In [31]:
print(feed.shapes[feed.shapes.shape_id == shape_id])
print(feed2.shapes[feed2.shapes.shape_id == shape_id])

                   shape_id  shape_pt_lat  shape_pt_lon  shape_pt_sequence  \
298698  290:747#23138#23171    -40.753249    175.140556                  0   
298699  290:747#23138#23171    -40.753169    175.140430                  1   
298700  290:747#23138#23171    -40.753757    175.140056                  2   
298701  290:747#23138#23171    -40.754599    175.139590                  3   
298702  290:747#23138#23171    -40.755603    175.139034                  4   
298703  290:747#23138#23171    -40.755781    175.138841                  5   
298704  290:747#23138#23171    -40.755803    175.138861                  6   
298705  290:747#23138#23171    -40.755781    175.138841                  7   
298706  290:747#23138#23171    -40.756106    175.138486                  8   
298707  290:747#23138#23171    -40.756784    175.137746                  9   
298708  290:747#23138#23171    -40.756919    175.137599                 10   
298709  290:747#23138#23171    -40.757116    175.137415         